In [ ]:
#How it works:
#After calling "calculate_averages" you have the teamAveragesList list.
#This is a list where each element in the list is a "TeamAverages" object.
#There is one of these objects for each team, so the list has roughly 40 elements
#The object has a teamName, and then some lists holding data.
#Each element in these lists represents a game, and holds the AVERAGE team performance up until and including that game (minus some exceptions)
#For example, if the team name is "Calgary Flames" and you look at the 10th element in the "GameGDs" list, you will find the avg GD of that team for the first 10 games.
#Finally, if you call the "retrieveGameAverages" function with a team name, a date (datetime object) and the list of team objects, it will return two dictionaries.
#The first dictionary holds the specific averages for that team on the given date. The second holds the specific averages for the opponent team on the given date.

import sys
import pandas as pd
import numpy as np
import datetime

gameData = pd.DataFrame()
class TeamAverages:
    teamName = ""
    gameOpponents = []
    gameDates = []
    gameLocations = []
    gameResults = []
    gamePoints = []
    gameGDs = []
    gameSDs = []
    gameSHs = []
    gameFOs = []
    gameSVs = []

    def __init__(self, name):
        self.teamName = name
        self.gameOpponents = []
        self.gameDates = []
        self.gameLocations = []
        self.gameResults = []
        self.gamePoints = []
        self.gameGDs = []
        self.gameSDs = []
        self.gameSHs = []
        self.gameFOs = []
        self.gameSVs = []
    
    def __eq__(self, other):
        return self.teamName == other

def main():
    filename = "GAME_LOGS.xlsx"
    gameSheets = pd.read_excel(filename, sheet_name=None)
    teamAveragesList = calculate_averages(gameSheets)

    #FOR TESTING    
    getGameData(teamAveragesList)
    gameData = createDataset(teamAveragesList)
    
    print(gameData)

def getGameData(teamAveragesList):
  global gameData
  gameData = createDataset(teamAveragesList)

def calculate_averages(teams):
    teamAveragesList = []
    for teamName in teams:
        teamDataGram = teams[teamName]
        teamDataGram = teamDataGram.dropna()
        newTeam = TeamAverages(teamName)
        totalGames = len(teamDataGram["Rank"])
        datesList = teamDataGram["Date"].tolist()
        opponentsList = teamDataGram["Opponent"].tolist()
        locationsList = convertLocationList(teamDataGram["Location"].tolist())
        resultsList = convertResultList(teamDataGram["Result"].tolist())
        pointsList = teamDataGram["Points"].tolist()
        gdList = teamDataGram["GD"].tolist()
        sdList = teamDataGram["SD"].tolist()
        shList = teamDataGram["SH%"].tolist()
        foList = teamDataGram["FO%"].tolist()
        svList = teamDataGram["SV%"].tolist()
        totalPoints = 0.0
        totalGD = 0.0
        totalSD = 0.0
        totalSH = 0.0 
        totalFO = 0.0
        totalSV = 0.0

        for gameNum in range(totalGames):
            totalPoints += pointsList[gameNum]
            totalGD += gdList[gameNum]
            totalSD += sdList[gameNum]
            totalSH += shList[gameNum]
            totalFO += foList[gameNum]
            totalSV += svList[gameNum]
            newTeam.gameDates.append(datesList[gameNum])
            newTeam.gameOpponents.append(opponentsList[gameNum])
            newTeam.gameLocations.append(locationsList[gameNum])
            newTeam.gameResults.append(resultsList[gameNum])
            newTeam.gamePoints.append((totalPoints / (gameNum + 1) / 2))
            newTeam.gameGDs.append(totalGD / (gameNum + 1))
            newTeam.gameSDs.append(totalSD / (gameNum + 1))
            newTeam.gameSHs.append(totalSH / (gameNum + 1))
            newTeam.gameFOs.append(totalFO / (gameNum + 1))
            newTeam.gameSVs.append(totalSV / (gameNum + 1))
        teamAveragesList.append(newTeam)
    return teamAveragesList

def convertResultList(resultsList):
    binaryList = []
    for result in resultsList:
        if (result == "Win" or result == "OT Win" or result == "SO Win"):
            binaryList.append(1)
        else:
            binaryList.append(0)
    return binaryList

def convertLocationList(locationsList):
    binaryList = []
    for location in locationsList:
        if (location == "Home"):
            binaryList.append(1)
        else:
            binaryList.append(0)
    return binaryList

def retrieveGameAverages(teamName, date, teamAveragesList):
    team = teamAveragesList[teamAveragesList.index(teamName)]
    dateIndex = team.gameDates.index(date)
    opponentTeamName = team.gameOpponents[dateIndex]
    opponentTeam = teamAveragesList[teamAveragesList.index(opponentTeamName)]
    opponentDateIndex = opponentTeam.gameDates.index(date)
    teamAverages = {"teamName": teamName}
    teamAverages["gameDate"] = date
    teamAverages["gameLocation"] = team.gameLocations[dateIndex]
    teamAverages["gameResult"] = team.gameResults[dateIndex]
    teamAverages["gamePoints"] = team.gamePoints[dateIndex]
    teamAverages["gameGD"] = team.gameGDs[dateIndex]
    teamAverages["gameSD"] = team.gameSDs[dateIndex]
    teamAverages["gameSH"] = team.gameSHs[dateIndex]
    teamAverages["gameFO"] = team.gameFOs[dateIndex]
    teamAverages["gameSV"] = team.gameSVs[dateIndex]
    opponentAverages = {"teamName": opponentTeamName}
    opponentAverages["gameDate"] = date
    opponentAverages["gameLocation"] = opponentTeam.gameLocations[opponentDateIndex]
    opponentAverages["gameResult"] = opponentTeam.gameResults[opponentDateIndex]
    opponentAverages["gamePoints"] = opponentTeam.gamePoints[opponentDateIndex]
    opponentAverages["gameGD"] = opponentTeam.gameGDs[opponentDateIndex]
    opponentAverages["gameSD"] = opponentTeam.gameSDs[opponentDateIndex]
    opponentAverages["gameSH"] = opponentTeam.gameSHs[opponentDateIndex]
    opponentAverages["gameFO"] = opponentTeam.gameFOs[opponentDateIndex]
    opponentAverages["gameSV"] = opponentTeam.gameSVs[opponentDateIndex]
    return teamAverages, opponentAverages


def createDataset(teamAverages):

    gameObject = [] #Will be list of tuples where each tuple is a row in the DataFrame
    games = [] 
    dates = []

    for teamAvg in teamAverages:
      for date in teamAvg.gameDates:
        if date not in dates:
          dates.append(date)

    # print(dates[0])
    # print(teamAverages[0].gameDates[0])
    # print(dates[0] == teamAverages[0].gameDates[0])
    # Creating a list of tuples
    #for date in TeamAverages.gameDates:


    # FOR A SPECIFIC DATE AND A SPECIFIC TEAM THAT PLAYED ON THAT DATE
    # WE RETRIEVE THE GAME AVERAGE DATA FOR IT AND USE IT
    # TO CREATE A TUPLE
    for date in dates:
      for teamAvg in teamAverages:
        if date in teamAvg.gameDates:
          #print(teamAvg.teamName)
          games.append(retrieveGameAverages(teamAvg.teamName, date, teamAverages))

    for game in games:
      
      if (game[0]['gameLocation'] == 1) :
        homeTeam = game[0]
        awayTeam = game[1]
      else:
        awayTeam = game[0]
        homeTeam = game[1]

      goalDiff = homeTeam['gameGD']
      shotDiff = homeTeam['gameSD']

      shotPercentDiff = homeTeam['gameSH'] - awayTeam['gameSH']
      foPercentDiff = homeTeam['gameFO'] - awayTeam['gameFO']
      savePercentDiff = homeTeam['gameSV'] - awayTeam['gameSV']
      pointsPercentDiff = homeTeam['gamePoints'] - awayTeam['gamePoints']
      homeResult = homeTeam['gameResult']

      gameTuple = (homeTeam['teamName'], awayTeam['teamName'], homeTeam['gameDate'], goalDiff, shotDiff, shotPercentDiff, foPercentDiff, savePercentDiff, pointsPercentDiff, homeResult)
      gameObject.append(gameTuple)




    
    dfObj = pd.DataFrame(gameObject,columns=['HomeTeam', 'AwayTeam', 'Date', 'GoalDiff', 'ShotDiff','ShotPercentDiff','foPercentDiff','savePercentDiff','pointsPercentDiff','HomeTeamResult'])


    return dfObj
    

    

main()

                 HomeTeam           AwayTeam  ... pointsPercentDiff  HomeTeamResult
0           Anaheim Ducks    Arizona Coyotes  ...          1.000000               1
1           Anaheim Ducks    Arizona Coyotes  ...          1.000000               1
2            Dallas Stars      Boston Bruins  ...         -1.000000               0
3     Pittsburgh Penguins     Buffalo Sabres  ...         -1.000000               0
4      Colorado Avalanche     Calgary Flames  ...          1.000000               1
...                   ...                ...  ...               ...             ...
2159      Ottawa Senators   New York Rangers  ...          0.000000               1
2160      Ottawa Senators   New York Rangers  ...          0.000000               1
2161  Pittsburgh Penguins  New Jersey Devils  ...          0.180124               1
2162  Pittsburgh Penguins    Ottawa Senators  ...          0.204167               1
2163  Pittsburgh Penguins    Ottawa Senators  ...          0.204167         

# Neural Network


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np



inputs = gameData.drop(columns=['Date','HomeTeam','AwayTeam','HomeTeamResult'])


target = gameData.HomeTeamResult


X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2, random_state=0, shuffle=False)#inputs = X, target = y
mlpc=MLPClassifier()
mlpc.fit(X_train, y_train)
pred_mlpc = mlpc.predict(X_test)
pred_mlpc_train = mlpc.predict(X_train)

print("Neural Network")

#Training Results
print("Training Accuracy:",metrics.accuracy_score(y_train, pred_mlpc_train))
print(confusion_matrix(y_train,pred_mlpc_train))
print(classification_report(y_train,pred_mlpc_train))

#Testing Results
print("Testing Accuracy:",metrics.accuracy_score(y_test, pred_mlpc))
print(confusion_matrix(y_test,pred_mlpc))
print(classification_report(y_test,pred_mlpc))

Neural Network
Training Accuracy: 0.6562680531484691
[[539 286]
 [309 597]]
              precision    recall  f1-score   support

           0       0.64      0.65      0.64       825
           1       0.68      0.66      0.67       906

    accuracy                           0.66      1731
   macro avg       0.66      0.66      0.66      1731
weighted avg       0.66      0.66      0.66      1731

Testing Accuracy: 0.6096997690531177
[[111  74]
 [ 95 153]]
              precision    recall  f1-score   support

           0       0.54      0.60      0.57       185
           1       0.67      0.62      0.64       248

    accuracy                           0.61       433
   macro avg       0.61      0.61      0.61       433
weighted avg       0.62      0.61      0.61       433



/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


**GridSearch**

In [ ]:
parameters = {'solver': ['lbfgs'], 'max_iter': [1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000 ], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':np.arange(10, 15), 'random_state':[0,1,2,3,4,5,6,7,8,9]}
clf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1, verbose=1)

clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.best_params_)


result = clf.predict(X_test)
print(result)

Fitting 5 folds for each of 4950 candidates, totalling 24750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   29.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 17.3min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 25.4min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed: 36.1min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 48.5min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 61.4min
[Parallel(n_jobs=-1)]: Done 4996 tasks      | elapsed: 78.3min
[Parallel(n_jobs=-1)]: Done 6046 tasks      | elapsed: 96.6min
[Parallel(n_jobs=-1)]: Done 7196 tasks      | elapsed: 115.3min
[Parallel(n_jobs=-1)]: Done 8446 tasks      | elapsed: 137.5min
[Parallel(n_jobs=-1)]: Done 9796 tasks      | elapsed: 159.3min
[Parallel(n_jobs=-1)]: Done 11246 tasks    

In [ ]:
import sys
from termcolor import colored, cprint

text = colored('WIN!', 'green')
vs = colored('VS', 'red')

#CREATE A TABLE OF OUTCOMES WIN/LOSS THAT CORRESPONDS TO 1/0
game_outcomes = np.where(pred_mlpc==1, "Team 1 wins","Team 2 wins")

#GET THE TEAMS FOR THE TEST DATA (LAST 20%)
team_1 = gameData.HomeTeam[1730:2163]
team_2 = gameData.AwayTeam[1730:2163]

#STACK THE COLUNS ON TOP OF EACH OTHER
games = np.vstack((team_1, team_2)).T
results = np.column_stack((games,game_outcomes))

#DISPLAY THE RESULTS
from termcolor import colored, cprint
for c in results[0:7]:
    if(c[2] == "Team 2 wins"):
      print(c[0] + " " +  vs + " " + c[1] + " = " + colored(c[1], 'green') + " " + text)
      print()
    else:
      print(c[0] + " " +  vs + " "  + c[1] + " = " + colored(c[0], 'green')+ " " +  text)
      print()

Ottawa Senators VS Montreal Canadiens = Montreal Canadiens WIN!

Nashville Predators VS Columbus Blue Jackets = Columbus Blue Jackets WIN!

New Jersey Devils VS Washington Capitals = New Jersey Devils WIN!

New York Rangers VS San Jose Sharks = San Jose Sharks WIN!

Ottawa Senators VS Montreal Canadiens = Ottawa Senators WIN!

Philadelphia Flyers VS Winnipeg Jets = Philadelphia Flyers WIN!

Pittsburgh Penguins VS Buffalo Sabres = Pittsburgh Penguins WIN!

